In [ ]:
!pip install -q kaggle

!kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset

!unzip real-life-violence-situations-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/mohamedmustafa/real-life-violence-situations-dataset
License(s): copyright-authors
100% 3.58G/3.58G [03:14<00:00, 16.1MB/s]
100% 3.58G/3.58G [03:14<00:00, 19.7MB/s]
Archive:  real-life-violence-situations-dataset.zip
  inflating: Real Life Violence Dataset/NonViolence/NV_1.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_10.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_100.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_1000.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_101.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_102.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_103.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_104.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_105.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_106.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_107.mp4  
  inflating: Real Life Vi

In [ ]:
HEIGHT , WIDTH = 224, 224
# number of frames of a video
SEQUENCE_LENGTH = 12

DATASET_DIR = "/content/real life violence situations/Real Life Violence Dataset"

CLASSES_LIST = ["NonViolence","Violence"]

In [ ]:
import cv2

# import imageio
import imgaug.augmenters as iaa
# import imgaug as ia

import os

import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import keras
from keras.layers import *
from keras.models import Sequential

#test
# frames_extraction("/content/real life violence situations/Real Life Violence Dataset/Violence/V_1.mp4")

In [ ]:
def frames_extraction(video_path):

    frames_list = []

    # Read the Video File
    video_reader = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):
        # set next frame.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading the frame from the video.
        success, image = video_reader.read()

        zoom = iaa.Affine(scale=1.3)
        random_brightness = iaa.Multiply((1, 1.3))

        image_aug = random_brightness(image = image)
        image_aug = zoom(image = image_aug)

        if not success:
            break

        # Resize
        resized_frame = cv2.resize(image_aug, (HEIGHT, WIDTH))

        # Normalize the resized frame
        normalized_frame = resized_frame / 255

        # Append to frames list
        frames_list.append(normalized_frame)


    video_reader.release()

    return frames_list

In [ ]:
def create_dataset():

    features = []
    labels = []
    # video_files_paths = []

    for class_index, class_name in enumerate(CLASSES_LIST):

        print(f'Extracting Data of Class: {class_name}')

        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))

        for file_name in files_list[:100]:

            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            frames = frames_extraction(video_file_path)

            if len(frames) == SEQUENCE_LENGTH:

                features.append(frames)
                labels.append(class_index)
                # video_files_paths.append(video_file_path)

    features = np.asarray(features)
    labels = np.array(labels)

    # return features, labels, video_files_paths
    return features, labels

In [ ]:
# Create the dataset.
# features, labels, video_files_paths = create_dataset()
features, labels = create_dataset()

Extracting Data of Class: NonViolence
Extracting Data of Class: Violence


In [ ]:
# Saving the extracted data
np.save("features.npy",features)
np.save("labels.npy",labels)
# np.save("video_files_paths.npy",video_files_paths)

# Load the saved files
# features = np.load("features.npy")
# labels = np.load("labels.npy")

In [ ]:
# convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [ ]:
# Split the Data into Train ( 80% ) and Test Set ( 20% ).
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.2, shuffle = True, random_state = 42)

In [ ]:
print(features_train.shape,labels_train.shape )
print(features_test.shape, labels_test.shape)

(160, 12, 224, 224, 3) (160, 2)
(40, 12, 224, 224, 3) (40, 2)


In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2
my_model = MobileNetV2(include_top = False , weights="imagenet")

#Fine-Tuning to make the last 40 layer trainable
my_model.trainable = False

<ipython-input-11-c077c14caead>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  my_model = MobileNetV2(include_top = False , weights="imagenet")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
def create_model():
    # LSTM => RNN
    # RNN => https://www.youtube.com/watch?v=rWGM2KRsPTg&ab_channel=AhmadAlseirafi
    # LSTM => https://www.youtube.com/watch?v=YD-rRmcacd8&ab_channel=AhmadAlseirafi
    lstm_fw = LSTM(units=32)
    lstm_bw = LSTM(units=32, go_backwards = True)
    optimizers = tf.keras.optimizers

    model = Sequential([

    ########################################################################################################################

    #Specifying Input to match features shape
    Input(shape = (SEQUENCE_LENGTH, HEIGHT, WIDTH, 3)),

    # Passing mobilenet in the TimeDistributed layer to handle the sequence
    TimeDistributed(my_model),
    Dropout(0.25),
    # Time distributed
    TimeDistributed(Flatten()),

    Bidirectional(lstm_fw, backward_layer = lstm_bw),
    Dropout(0.25),
        # Layer 1
    Dense(256,activation='relu'),
    Dropout(0.25),
    Dense(128,activation='relu'),
    Dropout(0.25),
        # Layer 3
    Dense(64,activation='relu'),
    Dropout(0.25),
        #  Layer 4
    Dense(32,activation='relu'),
    Dropout(0.25),
        # #Layer 1
        # softmax
    Dense(len(CLASSES_LIST), activation = 'sigmoid')
    ])

    adam = optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss="binary_crossentropy", optimizer=adam, metrics=["accuracy"])
    # Compiling the model
    #model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ["accuracy"])
    ########################################################################################################################

    model.summary()

    return model

In [ ]:
my_LSTM_model = create_model()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 12, 7, 7, 1280)      │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 12, 7, 7, 1280)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 12, 62720)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 64)                  │      16,064,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,382,690 (70.12 MB)

 Trainable params: 16,124,706 (61.51 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# Fitting the model
my_LSTM_model_history = my_LSTM_model.fit(x = features_train, y = labels_train, epochs = 20, batch_size = 8 ,
                                             shuffle = True, validation_split = 0.2)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.5600 - loss: 0.7046 - val_accuracy: 0.4688 - val_loss: 0.6919
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 254ms/step - accuracy: 0.5595 - loss: 0.6800 - val_accuracy: 0.5312 - val_loss: 0.6862
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 273ms/step - accuracy: 0.4834 - loss: 0.6949 - val_accuracy: 0.7188 - val_loss: 0.6788
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 259ms/step - accuracy: 0.6508 - loss: 0.6612 - val_accuracy: 0.7500 - val_loss: 0.6714
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 295ms/step - accuracy: 0.6991 - loss: 0.6587 - val_accuracy: 0.7500 - val_loss: 0.6370
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 291ms/step - accuracy: 0.7559 - loss: 0.6280 - val_accuracy: 0.8750 - val_loss: 0.5825
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 291ms/step - accuracy: 0.7546 - loss: 0.5469 - val_accuracy: 0.8125 - val_loss: 0.5190
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 279ms/step - accuracy: 0.8456 - loss: 0.4814 - val_accuracy: 0.7